<a href="https://colab.research.google.com/github/serggtech/Courses/blob/main/%D0%9A%D0%B0%D1%80%D1%82%D1%8B_%D0%91%D0%BB%D0%B5%D0%BA%D0%B4%D0%B6%D1%8D%D0%BA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Правила
Вначале игрок получает две карты «в открытую», сумма очков которых позволяет решить, нужны ему дополнительные карты или нет. Цель игры — набрать 21 очко или близкую к этому сумму. Если игрок набирает сумму очков, превышающую 21, то он проигрывает. Если сумма очков на картах дилера больше, чем 21, то игрок выигрывает.

Если сумма очков игрока равна сумме очков дилера, то объявляется «ничья».

Дилер набирает карты последним, при этом он обязан брать карту, если у него 16 очков или меньше, и остановиться, если сумма очков 17 или больше. Тузы считаются как 1 или как 11, «картинки» (валеты, дамы и короли) — все по 10 очков, остальные карты соответствуют своему номиналу.

In [ ]:
from random import shuffle

class Card:
    weights = {'J': 10, 'Q': 10, 'K': 10, 'A': 11}
    suits = ['♥', '♦', '♣', '♠']

    def __init__(self, rank, suit):
        self.rank = rank
        self.suit = suit
        self.weight = self.weights[rank] if rank in self.weights else int(rank)

    def __str__(self):
        return f"{self.rank}{self.suit}"

class Deck:
    suits = ['♥', '♦', '♣', '♠']
    ranks = ['2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K', 'A']

    def __init__(self, num_sets=1):
        self.cards = [Card(r, s) for _ in range(num_sets) for r in self.ranks for s in self.suits]
        self.__mix_deck()

    def __mix_deck(self):
        shuffle(self.cards)

    def take_card(self):
        if self.cards:
            return self.cards.pop()
        else:
            print("Deck is empty!")

class Player:
    def __init__(self, name):
        self.name = name
        self.hand = []

    def calculate_hand_value(self):
        value = sum(card.weight for card in self.hand)
        num_aces = sum(1 for card in self.hand if card.rank == 'A')

        while value > 21 and num_aces:
            value -= 10
            num_aces -= 1

        return value

class Dealer(Player):
    def __init__(self):
        super().__init__("Dealer")

    def show_first_card(self):
        print(f"{self.name}'s hand: {self.hand[0]}, ?")


class BlackjackGame:
    def __init__(self, player_name):
        self.player = Player(player_name)
        self.dealer = Dealer()
        self.deck = Deck()

    def start_game(self):
        print("Welcome to Blackjack!")
        self.player.hand = [self.deck.take_card(), self.deck.take_card()]
        self.dealer.hand = [self.deck.take_card(), self.deck.take_card()]
        self.show_game_status()

        while True:
            action = input("Choose your action: [H]it, [S]tand, [Q]uit: ").lower()
            if action == 'h':
                self.player_take_card()
                self.show_game_status()
                if self.check_player_bust():
                    break
            elif action == 's':
                self.dealer_play()
                break
            elif action == 'q':
                print("Thanks for playing! Exiting the game.")
                break
            else:
                print("Invalid input. Try again.")

    def player_take_card(self):
        card = self.deck.take_card()
        self.player.hand.append(card)
        print(f"You drew a {card}.")

    def dealer_play(self):
        print("\nDealer's turn:")
        self.dealer.show_first_card()

        while self.dealer.calculate_hand_value() < 17:
            card = self.deck.take_card()
            self.dealer.hand.append(card)
            print(f"Dealer drew a {card}.")
            self.show_game_status()

        if not self.check_player_bust():
            self.show_result()

    def check_player_bust(self):
        if self.player.calculate_hand_value() > 21:
            print("Bust! You went over 21. You lose.")
            self.show_result()
            return True
        return False

    def show_game_status(self):
        print(f"\n{self.player.name}'s hand: {', '.join(map(str, self.player.hand))} (Value: {self.player.calculate_hand_value()})")
        print(f"Dealer's hand: {self.dealer.hand[0]}, ?")

    def show_result(self):
        print(f"\n{self.player.name}'s hand: {', '.join(map(str, self.player.hand))} (Value: {self.player.calculate_hand_value()})")
        print(f"Dealer's hand: {', '.join(map(str, self.dealer.hand))} (Value: {self.dealer.calculate_hand_value()})")

        player_value = self.player.calculate_hand_value()
        dealer_value = self.dealer.calculate_hand_value()

        if player_value > 21:
            print("Bust! You went over 21. You lose.")
        elif dealer_value > 21:
            print("Dealer busts! You win!")
        elif player_value == dealer_value:
            print("It's a tie! Push.")
        elif dealer_value > player_value:
            print("Dealer wins!")
        else:
            print("You win!")


# Run the game
game = BlackjackGame("Player1")
game.start_game()

Welcome to Blackjack!

Player1's hand: J♣, Q♦ (Value: 20)
Dealer's hand: 3♥, ?
Choose your action: [H]it, [S]tand, [Q]uit: s

Dealer's turn:
Dealer's hand: 3♥, ?
Dealer drew a 2♣.

Player1's hand: J♣, Q♦ (Value: 20)
Dealer's hand: 3♥, ?
Dealer drew a 3♦.

Player1's hand: J♣, Q♦ (Value: 20)
Dealer's hand: 3♥, ?

Player1's hand: J♣, Q♦ (Value: 20)
Dealer's hand: 3♥, Q♣, 2♣, 3♦ (Value: 18)
You win!
